In [ ]:
def create_parameter_row(idx, parameters, language='es'):
    translations = {
        "es": {
            "fields": ["difusión", "reproducción", "expansión", "pendiente", "caminos"],
            "help_texts": ["Valor del coeficiente de {}."]
        },
        "en": {
            "fields": ["diffusion", "reproduction", "expansion", "slope", "paths"],
            "help_texts": ["Value of the {} coefficient."]
        },
        "pt": {
            "fields": ["difusão", "reprodução", "expansão", "inclinação", "caminhos"],
            "help_texts": ["Valor do coeficiente de {}."]
        }
    }

    cols = []

    for field, name in zip(FIELDS, translations[language]["fields"]):
        col = dbc.Col(
            dbc.InputGroup(
                [
                    dbc.InputGroupText(
                        help_text(field.title(), translations[language]["help_texts"][0].format(name))
                    ),
                    dbc.Input(
                        type="number",
                        required=True,
                        value=parameters[field],
                        placeholder=1,
                        min=1,
                        max=100,
                        id={
                            "type": "val-prediction-coefficient",
                            "field": field,
                            "index": idx,
                        },
                        debounce=True,
                    ),
                ],
                class_name="mb-2",
            ),
            width=3,
        )
        cols.append(col)

    cols.append(
        dbc.Col(
            dbc.Button(
                "⨉", class_name="bg-danger", id={"type": "btn-delete-row", "index": idx}
            ),
            width=1,
            class_name="text-center",
        )
    )
    return dbc.Row(cols, class_name="mb-2", id={"type": "row-parameters", "index": idx})